In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import numpy as np 
import glob 
import warnings 
from scipy.stats import norm
from IPython.display import display, HTML
import scipy
import os
warnings.filterwarnings("ignore")
%matplotlib inline
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
sns.set()

In [20]:
leagues = ['EPL','La_Liga','Bundesliga','Serie_A','Ligue_1','RFPL']

path = r'C:\Users\user\Desktop\Kaggle\Football stats analysis project\Data\UnderStat'

In [66]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [21]:
def getLeagueTeams(league):    
    dfOut =[]
    values=[]
    #Establish Connection
    driver.get('https://understat.com/league/'+league)
    #Get the relevant object
    base = driver.find_element_by_id('league-chemp')
    #Get Rows
    rows = base.find_elements_by_tag_name('tr')
    for item in range(1,len(rows)):
        #Get elements from the row
        row = rows[item].find_elements_by_tag_name('td')
        #Add those elements to the values list
        values.append([row[i].text for i in range(1,len(row))])
    dfOut = pd.DataFrame(values,columns=['team','matches','wins','draws','losses',
                                             'goals_for','goals_against','points','xG','xGA','xPTS'])
    return dfOut
    
def getLeagueStats(leagues):
    #Go through every league
    for league in leagues:
        df = getLeagueTeams(league)
        df['League'] = league
        df.to_csv(path + '\Team_Stats_%s'%league + '.csv')
    return print('Every league team stats crawled!')

getLeagueStats(leagues)    

Every league team stats crawled!


In [6]:
os.chdir(r'C:\Users\user\Desktop\Kaggle\Football stats analysis project\Data\UnderStat')
def loadData(path):
    allFiles = glob.glob(path)
    dfOut = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=0, header=0)
        list_.append(df)
    dfOut = pd.concat(list_)
    return dfOut
df = loadData('Team_Stats_*.csv')
team_names = df['team'].values
team_names = [i.replace(' ','_') for i in team_names]

In [22]:
"""Function mines players stats Data"""
def getPlayerStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    baseTable = driver.find_element_by_id('team-players')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns=['Name','Position','Apps','Min','Goals_Scored','Assists','Sh90',
                                     'KP90','xG','xA','xG90','xA90'])
    return df 


def getTeamPlayersStats(team_names):
    for name in team_names:
        df = getPlayerStats(name)
        df['Team'] = name
        df.to_csv(path + '\Player_Stats_%s'%name + '.csv')
    return print('Done')

getTeamPlayersStats(team_names)

Done


In [68]:
"""summary"""

def teamSummaryStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Situation','Sh','G','ShA','GA','xG','xGA','xGD','xG/Sh','xGA/Sh'])
    return df

"""formation"""

def teamFormationStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    driver.find_element_by_xpath("//label[@for='statistic-2']").click()
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Formation','Min','Sh','G','ShA','GA','xG','xGA','xGD','xG90','xGA90'])
    return df

"""Game State"""


def teamGameStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    driver.find_element_by_xpath("//label[@for='statistic-3']").click()
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Game_state','Min','Sh','G','ShA','GA','xG','xGA','xGD','xG90','xGA90'])
    return df

"""Timing"""


def teamTimingStats(name): 
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    driver.find_element_by_xpath("//label[@for='statistic-4']").click()
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Timing','Sh','G','ShA','GA','xG','xGA','xGD','xG/Sh','xGA/Sh'])
    return df

"""Shot Zones"""

def teamShotZonesStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    driver.find_element_by_xpath("//label[@for='statistic-5']").click()
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Shot_zones','Sh','G','ShA','GA','xG','xGA','xGD','xG/Sh','xGA/Sh'])
    return df

"""Attack speed"""


def teamAttackSpeedStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    driver.find_element_by_xpath("//label[@for='statistic-6']").click()
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Attack_speed','Sh','G','ShA','GA','xG','xGA','xGD','xG/Sh','xGA/Sh'])
    return df

"""Result"""

def teamFinalResultsStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    driver.find_element_by_xpath("//label[@for='statistic-7']").click()
    baseTable = driver.find_element_by_id('team-statistics')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns =['Result','Sh','G','ShA','GA','xG','xGA','xGD','xG/Sh','xGA/Sh'])
    return df


In [ ]:
def getTeamStats(team_names):
    for name in team_names:
        df_summary = teamSummaryStats(name)
        df_summary['Team'] = name
        df_formation = teamFormationStats(name)
        df_formation['Team'] = name
        df_gamestats = teamGameStats(name)
        df_gamestats['Team'] = name
        df_timing = teamTimingStats(name)
        df_timing['Team'] = name
        df_shotzones = teamShotZonesStats(name)
        df_shotzones['Team'] = name
        df_attackspeed = teamAttackSpeedStats(name)
        df_attackspeed['Team'] = name
        df_results = teamFinalResultsStats(name)
        df_results['Team'] = name
        return ('Done')


In [69]:
def getTeamStats(teams_names):
    for name in teams_names:
        df_summary = teamSummaryStats(name)
        df_summary['Team'] = name
        df_formation = teamFormationStats(name)
        df_formation['Team'] = name
        df_gamestats = teamGameStats(name)
        df_gamestats['Team'] = name
        df_timing = teamTimingStats(name)
        df_timing['Team'] = name
        df_shotzones = teamShotZonesStats(name)
        df_shotzones['Team'] = name
        df_attackspeed = teamAttackSpeedStats(name)
        df_attackspeed['Team'] = name
        df_results = teamFinalResultsStats(name)
        df_results['Team'] = name
        df = pd.concat([df_summary,df_timing,df_shotzones,df_attackspeed,df_results])
        df1 = pd.concat([df_formation,df_gamestats])
        df.to_csv(path + '\Team_Stats_1_%s'%name + '.csv')
        df1.to_csv(path + '\Team_Stats_2_%s'%name + '.csv')
        return ('Done')
    
teams_names = ['Manchester_United','Liverpool']
getTeamStats(teams_names)

'Done'